In [50]:
#Import Required Libraries
from PIL import Image
import requests

In [51]:
# Ensures that there is enough memory allocation for the model to load
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [52]:
from transformers import pipeline
import torch

pipe = pipeline(
    "image-text-to-text",
    model="google/gemma-3-4b-it",
    device="cuda",
    torch_dtype=torch.bfloat16
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.41it/s]
Device set to use cuda


In [ ]:
# Input Image

# Specify the path to your url file
file_path = '/home/jovyan/url.txt'  # Replace with your file's path

try:
    # Open the file in read mode
    with open(file_path, 'r') as file:
        # Read the contents of the file
        url = file.read()
        
        # Print the contents of the file
        print(questions)

except FileNotFoundError:
    print(f"Error: The file at {file_path} was not found.")
except IOError:
    print("Error: An error occurred while reading the file.")


image = Image.open(requests.get(url, stream=True).raw)
image 

In [54]:
# Number of questions to be generated 
questions = 20

In [55]:
# Specify the path to your .txt file
file_path = '/home/jovyan/questions(OD).txt'  # Replace with your file's path

try:
    # Open the file in read mode
    with open(file_path, 'r') as file:
        # Read the contents of the file
        questions = file.read()
        
        # Print the contents of the file
        print(questions)

except FileNotFoundError:
    print(f"Error: The file at {file_path} was not found.")
except IOError:
    print("Error: An error occurred while reading the file.")


 What is the main animal in the image?
 What type of clothing is the person wearing?
 What is on the ground near the person?
 What is the surface the person and dog are sitting on?
 What is the body of water visible in the background?
 What is the dog wearing?
 What is the person holding?
 What is the object near the dog's paw?
 What is the primary element in the background?
What is the general environment depicted?
What is the person doing?
What is the dog doing?
What is the object between the person and the dog?
What is the texture of the ground?
What is the shape of the waves?
What is the light source in the image?
What is the main feature of the dog’s appearance?
What is the person’s posture?
What is the color of the dog’s fur?
What is the main element in the foreground?


In [56]:
# Input prompt 
prompt = f"""
Task: Answer the input questions with short, concise answers given the input image.

Questions: {questions}

Include only the answers. Do not include the header.
"""
print(prompt)


Task: Answer the input questions with short, concise answers given the input image.

Questions:  What is the main animal in the image?
 What type of clothing is the person wearing?
 What is on the ground near the person?
 What is the surface the person and dog are sitting on?
 What is the body of water visible in the background?
 What is the dog wearing?
 What is the person holding?
 What is the object near the dog's paw?
 What is the primary element in the background?
What is the general environment depicted?
What is the person doing?
What is the dog doing?
What is the object between the person and the dog?
What is the texture of the ground?
What is the shape of the waves?
What is the light source in the image?
What is the main feature of the dog’s appearance?
What is the person’s posture?
What is the color of the dog’s fur?
What is the main element in the foreground?

Include only the answers. Do not include the header.



In [57]:
import torch
torch.cuda.empty_cache()

In [58]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "url": f"{url}"},
            {"type": "text", "text": f"{prompt}"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=1000)
print(output[0]["generated_text"][-1]["content"])
output_text = output[0]["generated_text"][-1]["content"]

Dog
Flannel shirt
Red leash
Sand
Ocean
Dog collar
Hand
Red toy
Paw
Sky
Beach
Sitting
Sitting
Nothing
Sandy
Wavy
Sun
Golden
Sitting


In [59]:
import re
import json 

questions_list = questions.strip().split('\n')
answers_list = output_text.strip().split('\n')
    
qa_data = [{"question": question, "answer": answer} for question, answer in zip(questions_list, answers_list)]  

# Step 4: Save the dictionary as a JSON file
file_name = 'q&a(gemma).json'
with open(file_name , 'w') as json_file:
    json.dump(qa_data, json_file, indent=4)

print(f"The question-answer pairs have been saved to {file_name}.")

# Open and load the JSON file
with open(file_name, 'r') as file:
    data = json.load(file)

# Print the contents of the JSON file
print(json.dumps(data, indent=4,ensure_ascii=False))  # This will format the JSON for better readability


The question-answer pairs have been saved to q&a(gemma).json.
[
    {
        "question": "What is the main animal in the image?",
        "answer": "Dog"
    },
    {
        "question": " What type of clothing is the person wearing?",
        "answer": "Flannel shirt"
    },
    {
        "question": " What is on the ground near the person?",
        "answer": "Red leash"
    },
    {
        "question": " What is the surface the person and dog are sitting on?",
        "answer": "Sand"
    },
    {
        "question": " What is the body of water visible in the background?",
        "answer": "Ocean"
    },
    {
        "question": " What is the dog wearing?",
        "answer": "Dog collar"
    },
    {
        "question": " What is the person holding?",
        "answer": "Hand"
    },
    {
        "question": " What is the object near the dog's paw?",
        "answer": "Red toy"
    },
    {
        "question": " What is the primary element in the background?",
        "answer": "